#  Latent Credible Analysis method on the weather data set 

## Introduction 

In this notebook, we will implement [*Latent Credible Analysis*](https://research.fb.com/publications/latent-credibility-analysis/) models. These are latent probablistic models that use hidden (latent) variables to represents the unknown data source reliabilities and underlying truth values. 

We implement only simpleLCA for now as extension to other models are relatively straight forward.



# SimpleLCA

Here is the plate model of simpleLCA. 

![simpleLCA](./gfx/simpleLCA.png)

## Importing packages 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os.path as op
import numpy as np
import seaborn as sns
import pyro

import sys
sys.path.insert(0, '../')

from LCA.preprocessing import encoders
from LCA.judge import lca, utils
from LCA import evaluator
import time

In [3]:
DATA_DIR = '../../../Data'

In [4]:
truths = pd.read_csv(op.join(DATA_DIR, 'weather/Claims_to_work_with/truths_with_encoding_INT.csv'))
claims = pd.read_csv(op.join(DATA_DIR,'weather/Claims_to_work_with/claims_with_encoding_INT.csv'))

In [5]:
truths.shape, claims.shape

((20905, 3), (278540, 5))

## Data Preprocessing 

We decide to model city population as discrete value. Moreover we consider the hidden truth value is only from the set of available assertions. Thus we need to label encode `value` of claims data frame.

We need to label encode values of objects in order to feed them to our simpleLCA model

In [6]:
#claims['value'] = claims['value'].apply(np.int64)

In [7]:
claims_enc, le_dict = encoders.transform(claims)

In [8]:
claims_enc

object        SourceID  value  object_id  \
0        sanfranciscothufeb402_Pressure             msn      0      18528   
1        sanfranciscothufeb402_Pressure          herald      0      18528   
2        sanfranciscothufeb402_Pressure  washingtonpost      0      18528   
3        sanfranciscothufeb402_Pressure        climaton      0      18528   
4        sanfranciscothufeb402_Pressure         nytimes      0      18528   
...                                 ...             ...    ...        ...   
278535  sanfranciscosunjan3120_Pressure   weatherforyou      0      18499   
278536  sanfranciscosunjan3120_Pressure          unisys      0      18499   
278537  sanfranciscosunjan3120_Pressure    wunderground      0      18499   
278538  sanfranciscosunjan3120_Pressure         foxnews      1      18499   
278539  sanfranciscosunjan3120_Pressure             cnn      0      18499   

        source_id  
0               6  
1               5  
2              10  
3               1  
4               7  
...           ...  
278535         13  
278536          8  
278537         14  
278538          4  
278539          2  

[278540 rows x 5 columns]

## Main program 

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import torch

import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist

pyro.set_rng_seed(101)

In [10]:
# svi = lca.bvi(lca.lca_model, lca.lca_guide, claims_enc, epochs=20, num_samples=1, learning_rate=1e-5)
svi = lca.bvi(lca.lca_model, lca.lca_guide, claims_enc, learning_rate=0.1, num_samples=1)

C:\Users\CBS Compter\anaconda3\envs\tensorflow7\lib\site-packages\pyro\infer\svi.py:53: FutureWarning: The `num_samples` argument to SVI is deprecated and will be removed in a future release. Use `pyro.infer.Predictive` class to draw samples from the posterior.
  'samples from the posterior.', FutureWarning)


In [11]:
losses = []

In [ ]:
losses.extend(lca.fit(svi, claims_enc, epochs=2))

In [ ]:
start=time.time()
discovered_truths = lca.discover_truths(posteriors=pyro.get_param_store())

We need to inverse transform the discovered truth value of each object into their original space.

In [ ]:
discovered_truths['value'] = discovered_truths.apply(lambda x: le_dict[x['object_id']].inverse_transform([x['value']])[0], axis=1)
end=time.time()

## Calculate the accuracy 

In [ ]:
accuracy =evaluator.accuracy(truths, discovered_truths)

## The final results 

In [ ]:
print(end-start)
print(accuracy)